In [1]:
from numpy.random import seed
seed(3363)
from tensorflow import set_random_seed
set_random_seed(3363)

In [2]:
from keras import regularizers
from keras.models import Sequential, load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Convolution2D, MaxPooling2D, Flatten, InputLayer, LeakyReLU, BatchNormalization, Dropout, GlobalAveragePooling2D
from keras.applications.densenet import DenseNet121
from sklearn.metrics import roc_auc_score
from dataset_batch import load_train_data, load_test_data
import tensorflow as tf

Using TensorFlow backend.


In [3]:
def as_keras_metric(method):
    import functools
    from keras import backend as K
    import tensorflow as tf
    @functools.wraps(method)
    def wrapper(self, args, **kwargs):
        """ Wrapper for turning tensorflow metrics into keras metrics """
        value, update_op = method(self, args, **kwargs)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([update_op]):
            value = tf.identity(value)
        return value
    return wrapper

class DenseNetModel():

    def __init__(self, input_dim=(224,224,3), output_dim=14, learning_rate=0.00001, epochs=5, drop_out=0.3,
                 freeze=True):

        # parms:
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.drop_out = drop_out
        self.trainable = not freeze

        # Define DenseNet
        self.model = Sequential()

        base_model = DenseNet121(weights='imagenet', include_top=False, pooling='avg', input_shape=self.input_dim)

        # Freeze the model first
        base_model.trainable = self.trainable

        self.model.add(base_model)

        self.model.add(Dropout(self.drop_out))
        self.model.add(Dense(512, kernel_regularizer=regularizers.l2(0.01)))
        self.model.add(Dropout(self.drop_out))
        self.model.add(Dense(96, kernel_regularizer=regularizers.l2(0.01)))
        self.model.add(Dropout(self.drop_out))
        self.model.add(Dense(self.output_dim, activation = 'sigmoid'))

        auc_roc = as_keras_metric(tf.metrics.auc)
        recall = as_keras_metric(tf.metrics.recall)
        precision = as_keras_metric(tf.metrics.precision)

        self.model.compile(optimizer = 'adam', loss = 'binary_crossentropy',
                           metrics = ['binary_accuracy', 'mae', auc_roc, recall, precision])
        self.model.summary()

        # Image augmentation
        self.core_idg = ImageDataGenerator(samplewise_center=True,
                              samplewise_std_normalization=True,
                              horizontal_flip=True,
                              vertical_flip=False,
                              height_shift_range=0.05,
                              width_shift_range=0.15,
                              rotation_range=10,
                              shear_range=0.2,
                              fill_mode='constant',
                              cval=1,
                              zoom_range=0.3)

    # Data standardization
    def standardize(self, x):
        self.core_idg.fit(x)
        return

    # fit the data
    def fit(self, x, y, vx, vy):
        print ("Start Training model")
        X_train, y_train, X_test, y_test = x, y, vx, vy
        hist = self.model.fit_generator(
            (self.core_idg.flow(X_train, y_train, batch_size = 20)),
            validation_data = self.core_idg.flow(X_test, y_test), epochs=self.epochs)
        print ("Done Training model")
        return hist

    # data did preprocessing            
    def inference(self, x):
        return self.model.predict(x)

    # make predicition
    def score(self, x, y):
        y_pred = self.predict(x)
        return roc_auc_score(y, y_pred, average = "macro")

    # pain data without preprocessing
    def predict(self, x):
        x = self.core_idg.standardize(x)
        return self.inference(x)

    def save_weight(self, path = 'baseline.h5'):
        print ("Start Saving model")
        self.model.save(path)
        print ("Done Saving model")
        return

    def load_weight(self, path = 'baseline.h5'):
        print ("Start Loading model")
        self.model.load_weights(path)
        print ("Done Loading model")
        return

In [4]:
if __name__ == "__main__":

    model = DenseNetModel(input_dim=(288,288,3), epochs=5, drop_out=0.65, freeze=True)

    X_vali, y_vali = load_train_data(min_cnt=0, max_cnt=1000)
    for itera in range(30):
        for base in range(1000, 10001, 1600):
            X_train, y_train = load_train_data(min_cnt=base, max_cnt=base+1599)
            model.standardize(X_train)
            model.fit(X_train, y_train, X_vali, y_vali)
        print("Iter",itera,"AVG AUC:",model.score(X_vali, y_vali))
        model.save_weight(path='baseline_30_'+str(itera)+'.h5')

    # Saving model
    model.save_weight()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Model)          (None, 1024)              7037504   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 96)                49248     
_________________________________________________________________
dropout_3 (Dropout)          (None, 96)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 14)                1358      
Total para

/home/owner/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2901: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


10.0 % done
15.0 % done
20.0 % done
25.0 % done
30.0 % done
35.0 % done
40.0 % done
45.0 % done
50.0 % done
55.0 % done
60.0 % done
65.0 % done
70.0 % done
75.0 % done
80.0 % done
85.0 % done
90.0 % done
95.0 % done
100.0 % done
Done Loading Training Images and Labels from 0 to 1000
(1001, 288, 288, 3) (1001, 14)
Loading Training Images and Labels
0.0 % done
5.0 % done
10.0 % done
15.0 % done
20.0 % done
25.0 % done
30.0 % done
35.0 % done
40.0 % done
45.0 % done
50.0 % done
55.0 % done
60.0 % done
65.0 % done
70.0 % done
75.0 % done
80.0 % done
85.0 % done
90.0 % done
95.0 % done
100.0 % done
Done Loading Training Images and Labels from 1000 to 2599
(1600, 288, 288, 3) (1600, 14)
Start Training model
Epoch 1/5
80/80 [==============================] - 41s 518ms/step - loss: 5.5699 - binary_accuracy: 0.8696 - mean_absolute_error: 0.1391 - auc: 0.5022 - recall: 0.9875 - precision: 0.0437 - val_loss: 3.4491 - val_binary_accuracy: 0.9486 - val_mean_absolute_error: 0.0531 - val_auc: 0.5490 

Epoch 5/5
80/80 [==============================] - 30s 379ms/step - loss: 0.2015 - binary_accuracy: 0.9501 - mean_absolute_error: 0.0970 - auc: 0.6769 - recall: 1.0000 - precision: 0.0494 - val_loss: 0.2025 - val_binary_accuracy: 0.9486 - val_mean_absolute_error: 0.0868 - val_auc: 0.6780 - val_recall: 1.0000 - val_precision: 0.0495
Done Training model
Loading Training Images and Labels
0.0 % done
5.0 % done
10.0 % done
15.0 % done
20.0 % done
25.0 % done
30.0 % done
35.0 % done
40.0 % done
45.0 % done
50.0 % done
55.0 % done
60.0 % done
65.0 % done
70.0 % done
75.0 % done
80.0 % done
85.0 % done
90.0 % done
95.0 % done
100.0 % done
Done Loading Training Images and Labels from 7400 to 8999
(1600, 288, 288, 3) (1600, 14)
Start Training model
Epoch 1/5
80/80 [==============================] - 33s 417ms/step - loss: 0.1916 - binary_accuracy: 0.9544 - mean_absolute_error: 0.0910 - auc: 0.6787 - recall: 1.0000 - precision: 0.0494 - val_loss: 0.2013 - val_binary_accuracy: 0.9486 - val_mean_ab

/home/owner/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2901: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


...

10.0 % done
15.0 % done
20.0 % done
25.0 % done
30.0 % done
35.0 % done
40.0 % done
45.0 % done
50.0 % done
55.0 % done
60.0 % done
65.0 % done
70.0 % done
75.0 % done
80.0 % done
85.0 % done
90.0 % done
95.0 % done
100.0 % done
Done Loading Training Images and Labels from 1000 to 2599
(1600, 288, 288, 3) (1600, 14)
Start Training model
Epoch 1/5
80/80 [==============================] - 32s 402ms/step - loss: 0.1844 - binary_accuracy: 0.9549 - mean_absolute_error: 0.0842 - auc: 0.7249 - recall: 1.0000 - precision: 0.0496 - val_loss: 0.2102 - val_binary_accuracy: 0.9486 - val_mean_absolute_error: 0.0855 - val_auc: 0.7250 - val_recall: 1.0000 - val_precision: 0.0496
Epoch 2/5
80/80 [==============================] - 30s 374ms/step - loss: 0.1841 - binary_accuracy: 0.9549 - mean_absolute_error: 0.0831 - auc: 0.7250 - recall: 1.0000 - precision: 0.0496 - val_loss: 0.2042 - val_binary_accuracy: 0.9486 - val_mean_absolute_error: 0.0902 - val_auc: 0.7250 - val_recall: 1.0000 - val_precision: 

Done Training model
Loading Training Images and Labels
0.0 % done
5.0 % done
10.0 % done
15.0 % done
20.0 % done
25.0 % done
30.0 % done
35.0 % done
40.0 % done
45.0 % done
50.0 % done
55.0 % done
60.0 % done
65.0 % done
70.0 % done
75.0 % done
80.0 % done
85.0 % done
90.0 % done
95.0 % done
100.0 % done
Done Loading Training Images and Labels from 7400 to 8999
(1600, 288, 288, 3) (1600, 14)
Start Training model
Epoch 1/5
80/80 [==============================] - 32s 402ms/step - loss: 0.1852 - binary_accuracy: 0.9544 - mean_absolute_error: 0.0854 - auc: 0.7253 - recall: 1.0000 - precision: 0.0496 - val_loss: 0.2019 - val_binary_accuracy: 0.9486 - val_mean_absolute_error: 0.0960 - val_auc: 0.7253 - val_recall: 1.0000 - val_precision: 0.0496
Epoch 2/5
80/80 [==============================] - 30s 371ms/step - loss: 0.1849 - binary_accuracy: 0.9544 - mean_absolute_error: 0.0849 - auc: 0.7253 - recall: 1.0000 - precision: 0.0496 - val_loss: 0.2025 - val_binary_accuracy: 0.9486 - val_mean_ab

/home/owner/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2901: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


5.0 % done
10.0 % done
15.0 % done
20.0 % done
25.0 % done
30.0 % done
35.0 % done
40.0 % done
45.0 % done
50.0 % done
55.0 % done
60.0 % done
65.0 % done
70.0 % done
75.0 % done
80.0 % done
85.0 % done
90.0 % done
95.0 % done
100.0 % done
Done Loading Training Images and Labels from 1000 to 2599
(1600, 288, 288, 3) (1600, 14)
Start Training model
Epoch 1/5
80/80 [==============================] - 32s 403ms/step - loss: 0.1834 - binary_accuracy: 0.9549 - mean_absolute_error: 0.0837 - auc: 0.7254 - recall: 1.0000 - precision: 0.0496 - val_loss: 0.2103 - val_binary_accuracy: 0.9486 - val_mean_absolute_error: 0.0799 - val_auc: 0.7254 - val_recall: 1.0000 - val_precision: 0.0496
Epoch 2/5
80/80 [==============================] - 30s 373ms/step - loss: 0.1823 - binary_accuracy: 0.9549 - mean_absolute_error: 0.0833 - auc: 0.7254 - recall: 1.0000 - precision: 0.0496 - val_loss: 0.2064 - val_binary_accuracy: 0.9486 - val_mean_absolute_error: 0.0949 - val_auc: 0.7254 - val_recall: 1.0000 - val_

MemoryError: 